### Цель этого ноутбука - проверить как работает библиотека hmmlearn, написанная на python

In [157]:
from hmmlearn import hmm
from hmmlearn import base
import numpy as np

#Проверять будем загружая сгенерированные данные одной моделью в другую модель в виде обучающей выборки
#В моем идеальном понимании обученная вторая модель по выбросам первой должна дать матрицу переходных вероятностей
#между состояниями очень близко похожую на матрицу этих вероятностей у первой модели


##Вводные данные:
#Модель скрытых марковских цепей всегда описывается тремя вещами: 
#transition matrix - матрица N x N переходных вероятностей между скрытыми состояниями
#emission matrix - матрица N x M вероятностей между скрытыми состояниями и наблюдаемыми выбросамими
#pi или startprob - вектор начальных вероятостей состояний, пока непонятно зачем, идем дальше

#transition matrix
tr =np.array([[0.8, 0.2],
          [0.3, 0.7]])

#emission matrix
em = np.array([[0.3,0.7,0.0,0.0],
              [0.0,0.0,0.1,0.9]])

#startprob vector
startpr = np.array([0.5,0.5])


#создаю модель Скрытых Марковских Цепей с дискретными выбросами
mod = hmm.MultinomialHMM(n_components=2, transmat_prior=tr, 
                        startprob_prior=startpr, n_iter=50, 
                         init_params='ste',
                         params='s')


#Интересная вещь, что emission матрицу нельзя задать при инициализации модели. 
#Почему? Мне непонятно. Но можно задать как атрибут.
mod.emissionprob_= em

#стартовый вектор тоже задается как атрибут
mod.startprob_=startpr

#смотрим, что модель нормально взяла данные, однако emission по прежнему не видно
print(mod)

MultinomialHMM(algorithm='viterbi', init_params='ste', n_components=2,
        n_iter=50, params='s', random_state=None,
        startprob_prior=array([0.5, 0.5]), tol=0.01,
        transmat_prior=array([[0.8, 0.2],
       [0.3, 0.7]]),
        verbose=False)


In [158]:
#если попытаться проверить модель встроенной функцией, то она выдаст ошибку, что нет атрибута transmat_
#Это вызывает вопросы, потому что на самом деле мы уже передали в модель эту матрицу, зачем
#ее еще раз прописывать как атрибут мне снова не понятно, но ладно

mod.transmat_ = tr
mod._check()

### Генерируем данные из первой готовой модели

In [159]:
#модель готова, теперь она может генерировать как скрытые сосотояния, так и наблюдения или выбросы
#произведем генерацию

emis = 10*[8]
hidden = 10*[9]

#генерим 10 пар массивов из скрытых состояний и наблюдений с длиной от 10 до 100
for i in range(0,10):
    emis[i], hidden[i] = mod.sample(10*(i+1))

    
#подготовка данных к нужному формату для обучения второй модели    
X = np.array(emis[0])
lengths = [len(array) for array in emis]

for array in emis:
    X = np.concatenate([X, array])

### Создаем вторую модель и обучаем ее

In [160]:
#Создаем вторую модель, котрая будет обучаться на данных из первой модели
#задаем количество скрытых состояний равное первой модели, т.е. 2, стартовый вектор

decodmod = hmm.MultinomialHMM(n_components=2, startprob_prior=startpr, 
                              n_iter=50,params='ste')


In [161]:
#!!!!!!!!!!!!!!!!!!Обучаем!!!!!!!!!!!!!!!!!!!!!!
decodmod.fit(X,lengths)

MultinomialHMM(algorithm='viterbi', init_params='ste', n_components=2,
        n_iter=50, params='ste',
        random_state=<mtrand.RandomState object at 0x10979ad38>,
        startprob_prior=array([0.5, 0.5]), tol=0.01, transmat_prior=1.0,
        verbose=False)

### Сравниваем результаты

In [162]:
print(decodmod.score(X), '\n')
print(decodmod.transmat_, '\n')
print(tr)

-570.5558487436765 

[[0.84228488 0.15771512]
 [0.28429487 0.71570513]] 

[[0.8 0.2]
 [0.3 0.7]]
